In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot  as plt
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd

AIS_path = r'.\Data'
AIS_data = []

# Iterate over files in the directory
for file_name in os.listdir(AIS_path):
    # Construct full file path
    file_path = os.path.join(AIS_path, file_name)
    
    # Check if the path is a file (not a directory)
    if os.path.isfile(file_path):
        # Open and read the file
        with open(file_path) as f:
            # Load JSON data from the file
            json_data = json.load(f)
            # Append loaded data to the list
            AIS_data.append(json_data)

# Normalize the JSON data
data_normalized = pd.json_normalize(AIS_data, 'data')

# Now you can work with your DataFrame 'data_normalized'
data_normalized.head(5)

# Read the polygon GeoJSON file
polygon = gpd.read_file('port_of_rdam.geojson')

# Ensure that CRS is properly defined for polygon
polygon.crs = 'EPSG:4326'

gdf = gpd.GeoDataFrame(
    data_normalized,
    geometry=gpd.points_from_xy(data_normalized['navigation.location.long'], data_normalized['navigation.location.lat']),
    crs=polygon.crs)  # Use the same CRS as the polygon

# Check if the points are within the polygon
result = gdf.geometry.within(polygon.unary_union)

# Add the in port status to the data
data_normalized['in_port'] = result